In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%capture
%pip install accelerate peft bitsandbytes
# %pip install -U flash-attn
# %pip install datasets

In [ ]:
hf_token = "hf_token"
from huggingface_hub import login

login(hf_token)

In [6]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

from sklearn.model_selection import train_test_split

2025-06-12 13:03:38.927316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749733419.126899      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749733419.182846      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'trl'

In [31]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-chat-hf"


# Running Simple Llama

In [32]:
model_tokenizer = AutoTokenizer.from_pretrained(base_model)


In [33]:
inputs = model_tokenizer("I live in Sydney NSW in Australia, tell me the places near sydney but not inside sydney where I can go for a day trip using public transport"
                  , return_tensors="pt"
                  ).to(device)
inputs

{'input_ids': tensor([[    1,   306,  5735,   297, 16198,  3865, 29956,   297,  8314, 29892,
          2649,   592,   278,  7600,  2978, 19205,  3801,   541,   451,  2768,
         19205,  3801,   988,   306,   508,   748,   363,   263,  2462, 17487,
           773,   970,  8608]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [34]:
# model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="auto")

In [35]:
# with torch.no_grad():
#     output_ids = model.generate(**inputs, top_k=10, do_sample=True)
# # output_ids[0] is logits
# generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print(generated_text)

# Dataset

In [36]:
ds1 = load_dataset("JasleenSingh91/travel-questions-response")
df1 = pd.DataFrame(ds1['train'])

In [37]:
x = 4897
print(df1.iloc[x]["Input"])
print("----------------------------------\n")
print(df1.iloc[x]["Response_1"])
print("----------------------------------\n")
print(df1.iloc[x]["Response_2"])

Recommend solo adventure destinations in Africa, including safaris, hiking to volcanoes, and exploring ancient cultures.
----------------------------------

Absolutely! Africa is a wonderful continent for solo adventures, offering a diverse range of experiences from adventure safaris, mountain treks, to exploring ancient cultures. Here are some recommendations:
1. Serengeti National Park, Tanzania: Known as the "Greatest Wildlife Show on Earth," the Serengeti is a must-visit destination for solo travelers who love wildlife. Go on a safari to witness the annual Great Migration, where millions of animals migrate in search of water and fresh grazing lands. Stay at a tented camp or lodge for an authentic African experience.
2. Mount Kilimanjaro, Tanzania: This is a once-in-a-lifetime adventure for solo travelers seeking a challenge. Mount Kilimanjaro is the highest peak in Africa and offers breathtaking views of the African landscape. Join a guided tour, hire a guide and porters, and tackl

In [38]:
df1.iloc[1]

Input         What activities and places should I explore in...
Response_1    I'd be happy to help you plan an exciting and ...
Response_2     Ahaha, I'd be thrilled to help you plan your ...
Name: 1, dtype: object

In [39]:
# making the system prompt text
system_prompt = "Can you please help me with."

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        system_msg = "Please help me with "
        user_input = self.df.iloc[index]["Input"]
        assistant_response = self.df.iloc[index]["Response_1"]
        return f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_input} [/INST]{assistant_response} </s>"

def format_row(user_input, assistant_response, system_msg=system_prompt):
    return f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_input} [/INST]{assistant_response} </s>"

df1["text"] = df1.apply(lambda row: format_row(row["Input"], row["Response_1"]), axis=1)

In [40]:
df1.head()

,Input,Response_1,Response_2,text
0,Can you suggest the best travel destinations f...,Absolutely! To help me suggest the best trave...,Hi there! I'd be delighted to help you plan y...,<s>[INST] <<SYS>>\nCan you please help me with...
1,What activities and places should I explore in...,I'd be happy to help you plan an exciting and ...,"Ahaha, I'd be thrilled to help you plan your ...",<s>[INST] <<SYS>>\nCan you please help me with...
2,Can you recommend relaxing destinations to vis...,Absolutely! Winter can be a beautiful and pea...,"As a travel agent, I'd be delighted to recomm...",<s>[INST] <<SYS>>\nCan you please help me with...
3,What are the top winter vacation spots?,Absolutely! I'd be happy to help you plan your...,"As a travel agent, I'm happy to share with yo...",<s>[INST] <<SYS>>\nCan you please help me with...
4,Can you recommend a great vacation destination...,"Absolutely! Based on your preferences, I woul...",I'd love to help you plan an unforgettable va...,<s>[INST] <<SYS>>\nCan you please help me with...


In [41]:
train_set, eval_set = train_test_split(df1, train_size=0.8, random_state=42)

In [42]:
print(train_set.shape)
print(eval_set.shape)

(4807, 4)
(1202, 4)


In [43]:
train_set

,Input,Response_1,Response_2,text
5634,Suggest a winter hiking destination with stunn...,I'd recommend Lake Tahoe in California and Ne...,A fantastic winter hiking destination with st...,<s>[INST] <<SYS>>\nCan you please help me with...
4434,Suggest a budget-friendly mountain Airbnb with...,Instruction: I'd be happy to help you find a b...,"Ah, I've got just the thing for you!\n\nI rec...",<s>[INST] <<SYS>>\nCan you please help me with...
2350,Recommend budget-friendly mountain resorts for...,I'd be happy to help you find some budget-fri...,"Ahaha, I'm excited to help you plan your next...",<s>[INST] <<SYS>>\nCan you please help me with...
5414,Recommend solo travel destinations for explori...,Subject: Exciting Solo Cave Exploring Adventur...,"Ah, an adventure-seeking solo traveler, eh? N...",<s>[INST] <<SYS>>\nCan you please help me with...
3303,Plan a fall trip to a destination known for it...,"Destination: Vermont, USA\n\nOverview:\nVermon...","Fall is indeed a wonderful time to travel, wi...",<s>[INST] <<SYS>>\nCan you please help me with...
...,...,...,...,...
3772,Recommend travel destinations offering both re...,Absolutely! I'd be happy to help you plan a va...,"As a travel agent, I'm excited to recommend t...",<s>[INST] <<SYS>>\nCan you please help me with...
5191,Recommend solo travel destinations in India of...,"Absolutely, I'd be happy to help you plan a sp...","Ah, thank you for considering India, a countr...",<s>[INST] <<SYS>>\nCan you please help me with...
5226,Suggest the best family vacation spots in Ital...,Italy is a wonderful destination for families ...,"Italy - the land of love, art, history, and d...",<s>[INST] <<SYS>>\nCan you please help me with...
5390,Recommend solo travel destinations in Greece f...,"Absolutely, I'd be happy to help you plan a so...","Welcome! As a travel agent, I'm thrilled to s...",<s>[INST] <<SYS>>\nCan you please help me with...


In [44]:
# train_set = MyDataset(train_set)
# eval_set = MyDataset(eval_set)

train_set = Dataset.from_pandas(train_set[["text"]])
eval_set = Dataset.from_pandas(eval_set[["text"]])

In [45]:
train_set[0]

{'text': "<s>[INST] <<SYS>>\nCan you please help me with.\n<</SYS>>\n\nSuggest a winter hiking destination with stunning landscapes and challenging trails. [/INST] I'd recommend Lake Tahoe in California and Nevada as a winter hiking destination. Known for its breathtaking landscapes, Lake Tahoe offers some of the most challenging and rewarding hikes during the winter months.\n\nPopular Trails:\n1. Cascade Falls Trail: A moderately difficult trail, featuring a stunning frozen waterfall and beautiful views of Lake Tahoe. (Approx. 5 Miles Roundtrip)\n2. Eagle Falls Trail: This challenging hike to Eagle Falls requires some ice axe and crampons usage, making it perfect for experienced hikers looking for a true winter adventure. (Approx. 5 Miles Roundtrip)\n3. Gladys Lake Trail: The trail leads to a picturesque lake that's surrounded by snowy peaks and frozen trees. Though the trail is challenging, the reward is worth the effort. (Approx. 7 Miles Roundtrip)\n\nWeather Conditions: Winter temp

# Model setup with LoRa 4 bit quantization

In [46]:
model=None
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    attn_implementation="flash_attention_2",
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
model = prepare_model_for_kbit_training(model)

# Training Config

In [90]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_params = SFTConfig(
    output_dir="/kaggle/working/output",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    save_steps=25,
    max_length = 500,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [91]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=eval_set,
    peft_config=peft_params,
    args=training_params,
    # max_length =1000,
    # packing=False,
)

Converting train dataset to ChatML:   0%|          | 0/4807 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/4807 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4807 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4807 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/1202 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/1202 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1202 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1202 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Training

In [50]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

trainer.train()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  11995 MiB |  12117 MiB |  33272 GiB |  33261 GiB |
|       from large pool |  11351 MiB |  11601 MiB |  33241 GiB |  33230 GiB |
|       from small pool |    644 MiB |    644 MiB |     30 GiB |     30 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  11995 MiB |  12117 MiB |  33272 GiB |  33261 GiB |
|       from large pool |  11351 MiB |  11601 MiB |  33241 GiB |

Step,Training Loss
25,1.137200
50,0.832400
75,0.777100


TrainOutput(global_step=75, training_loss=0.9155764770507813, metrics={'train_runtime': 11983.5164, 'train_samples_per_second': 0.401, 'train_steps_per_second': 0.006, 'total_flos': 8.560893240764006e+16, 'train_loss': 0.9155764770507813})

## Save Model and tokenizer

In [51]:
model.save_pretrained("/kaggle/working/output")

In [60]:
model_tokenizer.save_pretrained("/kaggle/working/tokenizer")

('/kaggle/working/tokenizer/tokenizer_config.json',
 '/kaggle/working/tokenizer/special_tokens_map.json',
 '/kaggle/working/tokenizer/tokenizer.model',
 '/kaggle/working/tokenizer/added_tokens.json',
 '/kaggle/working/tokenizer/tokenizer.json')

In [58]:
# !mkdir output
# !mkdir tokenizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
# trainer

In [61]:
# !zip -r /kaggle/working/tokenizer.zip /kaggle/working/tokenizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/tokenizer/ (stored 0%)
  adding: kaggle/working/tokenizer/special_tokens_map.json (deflated 74%)
  adding: kaggle/working/tokenizer/tokenizer.json (deflated 85%)
  adding: kaggle/working/tokenizer/tokenizer.model (deflated 55%)
  adding: kaggle/working/tokenizer/tokenizer_config.json (deflated 66%)


## Push to hugging face

In [98]:
from huggingface_hub import create_repo, upload_folder
repo_id="rakibulnahin/travel-chat-llama2-7b-lora-4bit-finetuned"
create_repo(repo_id, private=False)
upload_folder(
    repo_id=repo_id,
    folder_path="/kaggle/working/output/checkpoint-75",
    path_in_repo=".",
)

optimizer.pt:   0%|          | 0.00/269M [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rakibulnahin/travel-chat-llama2-7b-lora-4bit-finetuned/commit/2e00c8bed9dcb2ae9368e1e73bb3f7628880928a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2e00c8bed9dcb2ae9368e1e73bb3f7628880928a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rakibulnahin/travel-chat-llama2-7b-lora-4bit-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='rakibulnahin/travel-chat-llama2-7b-lora-4bit-finetuned'), pr_revision=None, pr_num=None)

## Inferencing model

In [11]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [40]:
import gc
import torch

del model  # or any large tensors/models
del base_model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [41]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # or torch.bfloat16 on newer GPUs
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map={"": 0}
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
# from peft import PeftModel
# from transformers import AutoModelForCausalLM
# device = "cuda" if torch.cuda.is_available() else "cpu"
# base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf").to(device)
model = PeftModel.from_pretrained(base_model, "rakibulnahin/travel-chat-llama2-7b-lora-4bit-finetuned").to(device)

In [43]:
tokenizer2 = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


In [45]:
model.eval()
prompt = "Make a travel plan for a exciting experience in Vietnam with friends including various activites and relaxing experience"
inputs = tokenizer2(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
print(response)

Make a travel plan for a exciting experience in Vietnam with friends including various activites and relaxing experience.
Title: Adventure-Filled Vietnam Excursion with Friends
Destination: Vietnam
Duration: 10 days
Purpose: To explore Vietnam's diverse culture, history, and natural beauty while enjoying various activities and relaxation experiences with friends.
Day 1: Arrival in Hanoi
* Arrive at Noi Bai International Airport in Hanoi and check into the hotel.
* Spend the day exploring the Old Quarter and Hoan Kiem Lake.
* Enjoy dinner at a local restaurant.

Day 2: Hanoi City Tour
* Visit the Temple of Literature, Ngoc Son Temple, and the Museum of Ethnology.
* Take a traditional cyclo tour of the city.
* Enjoy lunch at a local street food market.

Day 3: Ha Long Bay Day Trip
* Take an early morning bus to Halong Bay.
* Board a luxury cruise ship and spend the day exploring the bay's stunning limestone islands, caves, and waterfalls.
* Enjoy dinner onboard and overnight stay.

Day 4

In [73]:
model2.eval()

prompt = "make me a solo trip in a beach area"
inputs = tokenizer2(prompt, return_tensors="pt").to(model2.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
print(response)

make me a solo trip in a beach area. февраль [].
The beach area I have chosen for your solo trip is Bali, Indonesia. This island is known for its beautiful beaches, rich culture, and relaxing atmosphere. Here are some suggestions for activities to do during your solo travel:
1. Explore the local markets: Visit the traditional markets of Bali, such as Pasar Badung or Pasar Kumbasari, to experience the vibrant colors and scents of the island's unique culture. You can shop for souvenirs, try local foods, and interact with the friendly locals.
2. Take a yoga class: Bali is famous for its yoga retreats, and there are numerous studios throughout the island where you can take a class. Not only will this help you relax, but it will also provide an opportunity to meet other travelers and make new friends.
3. Go on a surfing adventure: Bali is known for its excellent surfing conditions, and there are many surf schools located along the coast. If you're a beginner, don't worry – there are instruc

In [74]:
prompt = "I want to travel to relaxing destination with affordable accomodation and relaxing sight"
inputs = tokenizer2(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

response = tokenizer2.decode(outputs[0], skip_special_tokens=True)
print(response)

I want to travel to relaxing destination with affordable accomodation and relaxing sightseeing activities. hopefully you can suggest some places that fit my criteria.
Certainly! Based on your preferences, here are some suggestions for destinations that offer affordable accommodation and relaxing sightseeing activities:
1. Bali, Indonesia: Bali is known as the "Island of the Gods" for its beautiful beaches, temples, and lush green landscapes. The island offers a range of accommodation options from budget-friendly guesthouses to luxury resorts. Some popular sightseeing activities include visiting the famous Tanah Lot Temple, exploring Ubud's art galleries, and relaxing on Kuta Beach. Prices vary depending on the season and location, but generally speaking, a budget of $50-$75 per day should cover most expenses.
2. Thailand: Thailand is another popular destination for those seeking a relaxing vacation at an affordable price. The country is home to numerous beautiful beaches, ancient templ

In [24]:
model
inputs

{'input_ids': tensor([[    1,  8561,   263,  9850,  3814,   363,   263,  5566, 11407,  7271,
           297, 18444,   411,  7875,  3704,  5164,  5039,  3246,   322, 26681,
           292,  7271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [84]:

# CANT DO THIS CUZ PIPELINE DOESNT SUPPORT LORA + QUANTIZATION
# from transformers import pipeline
# pipe = pipeline("text-generation", model="rakibulnahin/travel-chat-llama2-7b")